# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas as hvplot
import pandas as pd
import requests
from pprint import pprint


# Import API key
from api_keys import geoapify_key
geoapify_key

'1fbd509ca03d43e7ae963669e8629761'

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,3.37,70,3,3.17,GS,1699414670
1,1,jamestown,42.0970,-79.2353,5.77,80,100,5.66,US,1699414520
2,2,nemuro,43.3236,145.5750,9.02,66,47,7.91,JP,1699414770
3,3,west island,-12.1568,96.8225,28.99,74,40,5.14,CC,1699414678
4,4,sur,22.5667,59.5289,27.19,61,15,4.84,OM,1699414690


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo = True, 
    tiles = 'OSM',
    frame_width = 700, 
    frame_height = 500,
    size = 'Humidity',
    color = "City",
    alpha = 0.5
)



# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
test = city_data_df[city_data_df['Max Temp'] >=22]
test1 = test[test['Max Temp']<25]
test1 = test1.dropna(how='all')
test1.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,adamstown,-25.0660,-130.1015,22.68,88,100,8.22,PN,1699414668
6,6,port isabel,26.0734,-97.2086,24.45,82,0,4.12,US,1699414683
25,25,tame,6.4610,-71.7300,23.07,79,76,1.36,CO,1699415124
34,34,souillac,-20.5167,57.5167,22.21,100,75,0.00,MU,1699415126
38,38,inirida,3.8653,-67.9239,23.35,97,98,0.08,CO,1699415045


In [6]:
# Narrow down cities that fit criteria and drop any results with null values
small_city_data = city_data_df[city_data_df['Max Temp'] >=22]
small_city_data = small_city_data[small_city_data['Max Temp']<25]

# Drop any rows with null values
small_city_data = small_city_data.dropna(how='all')

# Display sample data
small_city_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,adamstown,-25.0660,-130.1015,22.68,88,100,8.22,PN,1699414668
6,6,port isabel,26.0734,-97.2086,24.45,82,0,4.12,US,1699414683
25,25,tame,6.4610,-71.7300,23.07,79,76,1.36,CO,1699415124
34,34,souillac,-20.5167,57.5167,22.21,100,75,0.00,MU,1699415126
38,38,inirida,3.8653,-67.9239,23.35,97,98,0.08,CO,1699415045


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = small_city_data.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,adamstown,PN,-25.0660,-130.1015,88,
6,port isabel,US,26.0734,-97.2086,82,
25,tame,CO,6.4610,-71.7300,79,
34,souillac,MU,-20.5167,57.5167,100,
38,inirida,CO,3.8653,-67.9239,97,
...,...,...,...,...,...,...
557,tchintabaraden,NE,15.8969,5.7985,15,
563,alaghsas,NE,17.0187,8.0168,16,
566,cafayate,AR,-26.0730,-65.9761,61,
577,senanga,ZM,-16.1167,23.2667,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set the parameters for the type of place
categories= "accommodation.hotel"

# Set the parameters for the type of search
radius = 10000
limit = 20
# set up a parameters dictionary
params = {
    "categories": categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    long = row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f"circle:{long},{lat},{radius}"
    params['bias'] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
adamstown - nearest hotel: No hotel found
port isabel - nearest hotel: Queen Isabel Inn
tame - nearest hotel: hotel libertadores
souillac - nearest hotel: Hotel SkylineInn
inirida - nearest hotel: Residencias El Turista del Guainia
san jorge - nearest hotel: Hotel Monumental
avarua - nearest hotel: Paradise Inn
mouila - nearest hotel: Le Lac Bleu
tabou - nearest hotel: hôtel le rochet
medina - nearest hotel: The Oberoi Madina
saint-philippe - nearest hotel: Le Baril
pitimbu - nearest hotel: Pousada Brisa Mar
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
fonte boa - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
hwange - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
celestun - nearest hotel: Hotel Flamingo Playa
nova sintra - nearest hotel: Residência Ka Dencho
cooper's town - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
farrukhnagar - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
5,adamstown,PN,-25.0660,-130.1015,88,No hotel found
6,port isabel,US,26.0734,-97.2086,82,Queen Isabel Inn
25,tame,CO,6.4610,-71.7300,79,hotel libertadores
34,souillac,MU,-20.5167,57.5167,100,Hotel SkylineInn
38,inirida,CO,3.8653,-67.9239,97,Residencias El Turista del Guainia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot

map_plot_2 = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo = True, 
    tiles = 'OSM',
    frame_width = 700, 
    frame_height = 500,
    size = 'Humidity',
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)